In [1]:
from __future__ import print_function
import os
import sys
import math
import pickle
import boto3
import os
import numpy as np
import kg
import encoding
import pandas as pd
from tqdm import tqdm
import time
import argparse

tqdm.pandas()
# pandarallel.initialize(progress_bar=True)
# bucket = os.environ.get("BUCKET_NAME", " ")
# raw_data_folder = os.environ.get("RAW_DATA", " ")

s3client = boto3.client('s3')

Using backend: pytorch


sys path is ['/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python36.zip', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/lib-dynload', '', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/dglke-0.1.0.dev0-py3.6.egg', '/home/ec2-user/workplace/Aws-gcr-sc-recommender-system/src/offline/docker/batch/container/batch/src/fasthan', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ec2-user/.ipython']


In [2]:
########################################
# 从s3同步数据
########################################
s3client = boto3.client('s3')


def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))


def write_to_s3(filename, bucket, key):
    print("upload s3://{}/{}".format(bucket, key))
    with open(filename, 'rb') as f:  # Read in binary mode
        # return s3client.upload_fileobj(f, bucket, key)
        return s3client.put_object(
            ACL='bucket-owner-full-control',
            Bucket=bucket,
            Key=key,
            Body=f
        )

def write_str_to_s3(content, bucket, key):
    print("write s3://{}/{}, content={}".format(bucket, key, content))
    s3client.put_object(Body=str(content).encode("utf8"), Bucket=bucket, Key=key, ACL='bucket-owner-full-control')

default_bucket = 'aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887'
default_prefix = 'sample-data'
parser = argparse.ArgumentParser()
parser.add_argument('--bucket', type=str, default=default_bucket)
parser.add_argument('--prefix', type=str, default=default_prefix)
args, _ = parser.parse_known_args()
bucket = args.bucket
prefix = args.prefix

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)
# 行为/物品数据同步
file_name_list = ['action.csv']
s3_folder = '{}/system/user-data'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
file_name_list = ['item.csv']
s3_folder = '{}/system/item-data'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
# 行为/物品数据加载
df_filter_action = pd.read_csv('info/action.csv')

bucket=aws-gcr-rs-sol-workshop-ap-southeast-1-522244679887
prefix='sample-data'
file preparation: download src key sample-data/system/item-data/item.csv to dst key info/item.csv


In [3]:
df_filter_item = pd.read_csv('info/item.csv',sep='_!_',names=['news_id','type_code','type','title','keywords','popularity','new'])

/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [4]:
df_filter_item.head()

,news_id,type_code,type,title,keywords,popularity,new
0,6552418723179790856,102,news_entertainment,谢娜三喜临门何炅送祝福吴昕送祝福只有沈梦辰不一样,"杜海涛,谢娜,何炅,沈梦辰,吴昕,快本",3,0
1,6552390851157295629,102,news_entertainment,杨幂景甜徐冬冬唐嫣不好好穿衣却美的有趣又撩人,"杨幂,徐冬冬,背带裙,大唐荣耀,唐嫣,景甜",9,0
2,6552309039697494532,103,news_sports,亚洲杯夺冠赔率日本伊朗领衔中国竟与泰国并列,"土库曼斯坦,乌兹别克斯坦,亚洲杯,赔率,小组赛",2,0
3,6552452056043487748,103,news_sports,马夏尔要去切尔西可以商量不过穆里尼奥的要价是4000万加威廉,"威廉,曼联,穆里尼奥,布莱顿,马夏尔",2,0
4,6552466727995703815,103,news_sports,昔日中超金靴半场独造6球虐爆辽足华夏送走他后悔吗,"阿洛,阿洛伊西奥,华夏幸福,埃尔纳内斯,穆里奇",0,0


In [ ]:
df_filter_action = pd.read_csv('info/action.csv',sep='_!_')

In [ ]:
df_filter_action.head()

In [5]:
# prepare model for batch process
os.environ['GRAPH_BUCKET'] = 'sagemaker-us-east-1-002224604296'
os.environ['KG_DBPEDIA_KEY'] = 'recommender-system-data/model/sort/content/words/mapping/kg_dbpedia.txt'
os.environ['KG_ENTITY_KEY'] = 'recommender-system-data/model/sort/content/words/mapping/entities_dbpedia.dict'
os.environ['KG_RELATION_KEY'] = 'recommender-system-data/model/sort/content/words/mapping/relations_dbpedia.dict'
os.environ['KG_ENTITY_INDUSTRY_KEY'] = 'recommender-system-data/model/sort/content/words/mapping/entity_industry.txt'
os.environ['KG_VOCAB_KEY'] = 'recommender-system-data/model/sort/content/words/mapping/vocab.json'
os.environ['DATA_INPUT_KEY'] = ''
os.environ['TRAIN_OUTPUT_KEY'] = 'recommender-system-data/model/sort/content/kg/news/gw/'
kg_path = os.environ['GRAPH_BUCKET']
dbpedia_key = os.environ['KG_DBPEDIA_KEY']
entity_key = os.environ['KG_ENTITY_KEY']
relation_key = os.environ['KG_RELATION_KEY']
entity_industry_key = os.environ['KG_ENTITY_INDUSTRY_KEY']
vocab_key = os.environ['KG_VOCAB_KEY']
data_input_key = os.environ['DATA_INPUT_KEY']
train_output_key = os.environ['TRAIN_OUTPUT_KEY']

env = {
    'GRAPH_BUCKET': kg_path,
    'KG_DBPEDIA_KEY': dbpedia_key,
    'KG_ENTITY_KEY': entity_key,
    'KG_RELATION_KEY': relation_key,
    'KG_ENTITY_INDUSTRY_KEY': entity_industry_key,
    'KG_VOCAB_KEY': vocab_key,
    'DATA_INPUT_KEY': data_input_key,
    'TRAIN_OUTPUT_KEY': train_output_key
}
graph = kg.Kg(env)  # Where we keep the model when it's loaded
model = encoding.encoding(graph, env)

loading vocabulary file /home/ec2-user/.fastNLP/fasthan/fasthan_base/vocab.txt
Load pre-trained BERT parameters from file /home/ec2-user/.fastNLP/fasthan/fasthan_base/model.bin.


In [6]:
len(df_filter_item)

2660

In [7]:
# generate dict_id_keywords for tfidf
dict_keywords_id = {}
for row in df_filter_item.iterrows():
    item_row = row[1]
    program_id = str(item_row['news_id'])
    for kw in item_row['keywords'].split(','):
        if kw not in dict_keywords_id.keys():
            dict_keywords_id[kw] = [program_id]
            continue
        current_list = dict_keywords_id[kw]
        current_list.append(program_id)
        dict_keywords_id[kw].append(program_id)
n_keyword_whole = len(dict_keywords_id)

In [19]:
def get_tfidf(category_property):
    if not category_property or str(category_property).lower() in ['nan', 'nr', '']:
        return [None]
    if not category_property:
        return [None]
    value = [item.strip() for item in category_property.split(',')]
    keywords_tfidf = {}
    for keyword in value:
        current_score = 1 / len(value)*math.log(n_keyword_whole / len(dict_keywords_id[keyword]))
        keywords_tfidf[keyword] = current_score
    return keywords_tfidf
        
def get_category(category_property):
    if not category_property or str(category_property).lower() in ['nan', 'nr', '']:
        return [None]
    if not category_property:
        return [None]
    return [item.strip().lower() for item in category_property.split(',')]
            
def get_single_item(item):
    if not item or str(item).lower().strip() in ['nan', 'nr', '']:
        return [None]
    return [str(item).lower().strip()]

def get_entities(title):
    return model[title]

def single_dict(raw_dict, feat, item_id):
    if feat not in raw_dict.keys():
        raw_dict[feat] = [item_id]
    else:
        current_list = raw_dict[feat]
        current_list.append(item_id)
        raw_dict[feat] = current_list

def list_dict(raw_dict, feat_list, item_id):
    for feat in feat_list:
        single_dict(raw_dict, feat, item_id)

def update_popularity(item_df, action_df):
    
        
def sort_by_score(df):
    logging.info("sort_by_score() enter, df.columns: {}".format(df.columns))
    df['popularity'].fillna(0, inplace=True)

    df['popularity_log'] = np.log1p(df['popularity'])
    popularity_log_max = df['popularity_log'].max()
    popularity_log_min = df['popularity_log'].min()

    df['popularity_scaled'] = ((df['popularity_log'] - popularity_log_min) / (
            popularity_log_max - popularity_log_min)) * 10

    df_sorted = df_with_score.sort_values(by='popularity_scaled', ascending=False)
    
    df_sorted = df_sorted.drop(
        ['popularity_log', 'popularity_scaled'], axis=1)

    logging.info("sort_by_score() return, df.columns: {}".format(df_sorted.columns))
    return df_sorted
        
def gen_news_id_news_property_dict(df):
    update_popularity(df)
    sort_by_score(df)
    
    news_id_news_property_dict = {}
    news_type_news_ids_dict = {}
    news_keywords_news_ids_dict = {}
    news_entities_news_ids_dict = {}
    news_words_news_ids_dict = {}
    
    for row in df.iterrows():
        item_row = row[1]
        program_id = str(item_row['news_id'])
        current_entities = get_entities(item_row['title'])[0]
        current_words = get_entities(item_row['title'])[1]
        program_dict = {
            'title': get_single_item(item_row['title']),
            'type': get_single_item(item_row['type']),
            'keywords': get_category(item_row['keywords']),
            'tfidf': get_tfidf(item_row['keywords']),
            'entities': current_entities,
            'words': current_words
        }
        news_id_news_property_dict[program_id] = program_dict
        list_dict(news_type_news_ids_dict, program_dict['type'], program_id)
        list_dict(news_keywords_news_ids_dict, program_dict['keywords'], program_id)
        list_dict(news_entities_news_ids_dict, program_dict['entities'], program_id)
        list_dict(news_words_news_ids_dict, program_dict['words'], program_id)

    result_dict = {
        'news_id_news_property_dict': news_id_news_property_dict,
        'news_type_news_ids_dict': news_type_news_ids_dict,
        'news_keywords_news_ids_dict': news_keywords_news_ids_dict,
        'news_entities_news_ids_dict': news_entities_news_ids_dict,
        'news_words_news_ids_dict': news_words_news_ids_dict
    }
    return result_dict

In [20]:
rd = gen_news_id_news_property_dict(df_filter_item)

In [12]:


def gen_news_properties_to_news_ids_dict(df):
    df_sorted = sort_by_score(df)
# 1. news_id_news_property_dict.pickle
# 2. news_type_news_ids_dict.pickle
# 3. news_keywords_news_ids_dict.pickle
# 4. news_entities_news_ids_dict.pickle
# 5. news_words_news_ids_dict.pickl
    news_type_news_ids_dict = {}
    news_keywords_news_ids_dict = {}
    news_entities_news_ids_dict = {}
    news_words_news_ids_dict = {}

    for row in df_sorted.iterrows():
        item_row = row[1]
        # program_id = {"id": item_row['program_id'], "score": item_row['cal_score'] }
        program_id = item_row['news_id']
        for key in [item for item in get_single_item(item_row['type']) if item is not None]:
            news_type_news_ids_dict.setdefault(key, []).append(program_id)

        for key in [item for item in get_category(item_row['keywords']) if item is not None]:
            news_keywords_news_ids_dict.setdefault(key, []).append(program_id)
        
        for key in [item for item in get_category(item_row['entities']) if item is not None]:
            news_entities_news_ids_dict.setdefault(key, []).append(program_id)
                    
        for key in [item for item in get_category(item_row['words']) if item is not None]:
            news_words_news_ids_dict.setdefault(key, []).append(program_id)

    result_dict = {
        'news_type_news_ids_dict': news_type_news_ids_dict,
        'news_keywords_news_ids_dict': news_keywords_news_ids_dict,
        'news_entities_news_ids_dict': news_entities_news_ids_dict,
        'news_words_news_ids_dict': news_words_news_ids_dict
    }

    return result_dict

In [ ]:
dicts_2 = gen_news_properties_to_news_ids_dict(df)

In [11]:
n = 0
for k, v in rd['news_id_news_property_dict'].items():
    print("k {} v {}".format(k,v))
    if n > 10:
        break
    n = n + 1

k 6552418723179790856 v {'title': ['谢娜三喜临门何炅送祝福吴昕送祝福只有沈梦辰不一样'], 'type': ['news_entertainment'], 'keywords': ['杜海涛', '谢娜', '何炅', '沈梦辰', '吴昕', '快本'], 'tfidf': {'杜海涛': 1.1915032285682567, '谢娜': 0.8605173146234215, '何炅': 0.9546056150797302, '沈梦辰': 1.3327195386327908, '吴昕': 1.1915032285682567, '快本': 1.1161723746110805}, 'entities': [40191, 0, 46990, 1871, 5802, 162743, 1871, 5802, 315, 390701, 28, 302, 0, 0, 0, 0], 'words': [559632, 0, 613175, 0, 0, 754092, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
k 6552390851157295629 v {'title': ['杨幂景甜徐冬冬唐嫣不好好穿衣却美的有趣又撩人'], 'type': ['news_entertainment'], 'keywords': ['杨幂', '徐冬冬', '背带裙', '大唐荣耀', '唐嫣', '景甜'], 'tfidf': {'杨幂': 1.1161723746110805, '徐冬冬': 1.3327195386327908, '背带裙': 1.5158215867441425, '大唐荣耀': 1.5158215867441425, '唐嫣': 1.3327195386327908, '景甜': 1.3327195386327908}, 'entities': [20585, 130577, 193876, 71718, 28, 2798, 20784, 382, 727, 2, 5876, 121, 67692, 0, 0, 0], 'words': [0, 0, 359872, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}
k 6552309039697494532 v {'ti

In [ ]:
def gen_pickle_files(df_filter_action, df_filter_content, out_s3_path):
    logging.info(f"gen_pick_files(), "
                 f"\nbasic_s3_path={basic_s3_path}, "
                 f"\nexpend_s3_path={expend_s3_path}, "
                 f"\nout_s3_path={out_s3_path}")

    dicts_1 = gen_movie_id_movie_property_dict(df_filter_action)
    dicts_2 = gen_movie_properties_to_movie_ids_dict(df)
    dicts_all = dicts_1
    dicts_all.update(dicts_2)
    bucket, out_prefix = get_bucket_key_from_s3_path(out_s3_path)
    for dict_name, dict_val in dicts_all.items():
        file_name = f'{dict_name}.pickle'
        # print("pickle =>", file_name)
        out_file = open(file_name, 'wb')
        pickle.dump(dict_val, out_file)
        out_file.close()
        # s3_url = S3Uploader.upload(file_name, out_s3_path)
        s3_url = write_to_s3(file_name, bucket, f'{out_prefix}/{file_name}')
        logging.info("write {}".format(s3_url))
    logging.info(f"generated {len(dicts_all)} pickle files")

In [ ]:
gen_pickle_files(df_filter_action, df_filter_content, out_s3_path)